In [1]:
# This file normalize dataframe by min max normalization

import pandas as pd
import numpy as np
import time
import os 
from pandas import Series, DataFrame
from tqdm import tqdm_notebook

dir_ = '../data/'
file_name = 'filter_track_5_user_100.pkl'

In [2]:
lh = pd.read_pickle(os.path.join(dir_, file_name))

In [3]:
unique_users = lh['uid'].unique()
unique_user = len(unique_users)
unique_tracks = lh['tid'].unique()
unique_track = len(unique_tracks)

print(unique_user)
print(unique_track)

120
26438


In [4]:
lh

,uid,tid,count
0,0,0,1
3,0,3,1
4,0,4,1
5,0,5,8
8,0,8,2
...,...,...,...
593264,125,178271,1
593306,125,190150,1
593314,125,191625,3
593336,125,205479,3


In [5]:
track_keys = {}

for track in unique_tracks:
    track_keys[track] = len(track_keys)
    
user_keys = {}

for user in unique_users:
    user_keys[user] = len(user_keys)

In [6]:
lh['tid']= lh['tid'].map(track_keys)
lh['uid']= lh['uid'].map(user_keys)

In [7]:
lh

,uid,tid,count
0,0,0,1
3,0,1,1
4,0,2,1
5,0,3,8
8,0,4,2
...,...,...,...
593264,119,26387,1
593306,119,26409,1
593314,119,26415,3
593336,119,26430,3


In [8]:
threshold = []
for i in tqdm_notebook(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    user = user.sort_values(by=['count'])
    t = []
    for j in range(19):
        v = user.iloc[len(user)//20*(j+1)]['count']
        t.append(v)
    threshold.append(t)
threshold

/home/han/anaconda3/envs/ts/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 5, 7, 12],
 [1, 1, 1, 2, 2, 3, 3, 4, 5, 6, 8, 9, 11, 12, 15, 18, 21, 26, 39],
 [1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 4, 5, 6, 8, 9, 12, 15, 22, 33],
 [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 4, 5, 7],
 [1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 7, 9, 13, 20, 39],
 [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 5, 6, 8, 11],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 6],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 5],
 [1, 1, 1, 1, 1, 2, 2, 3, 3, 4, 5, 6, 8, 10, 14, 17, 22, 31, 42],
 [1, 1, 1, 1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 11, 13, 17, 23],
 [1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 7, 8, 10, 12, 15],
 [1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 6, 7, 9, 11, 17],
 [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 6, 7, 8, 10, 12, 18],
 [1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 7, 9, 11, 16, 21, 31],
 [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 5, 6, 8],
 [

In [9]:
a = threshold[0][18]
print(a)

12


In [10]:
start = time.time()
def to_rating(df):  
    rating = 0
    for i in range(18,-1,-1):
        if df['count'] <= threshold[df['uid']][i]:
            continue
        else:
            rating = i+1
            break
#     print(df['uid'],df['count'],rating)
    return 0.25*(rating+1)
        
lh['rating'] = lh.apply(to_rating , axis = 1)
lh = lh[['uid','tid','rating','count']]

end = time.time()
print(end - start)

54.63713836669922


In [11]:
lh

,uid,tid,rating,count
0,0,0,0.25,1
3,0,1,0.25,1
4,0,2,0.25,1
5,0,3,4.75,8
8,0,4,2.25,2
...,...,...,...,...
593264,119,26387,0.25,1
593306,119,26409,0.25,1
593314,119,26415,2.50,3
593336,119,26430,2.50,3


In [12]:
lh.to_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))

In [13]:
len(lh[lh['rating']>1])

162166

In [14]:
lc = lh[lh['rating']>1]
print(len(lc))
lc2 = lc[lc['count']>4]
print(len(lc2)/len(lc))

162166
0.4403327454583575
